In [ ]:
import torch
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self, num_classes=200, feature_dim=256, K=128):
        super(MyModel, self).__init__()
        self.K = K
        self.num_classes = num_classes
        self.feature_dim = feature_dim

        self.register_buffer("u_bank", torch.randn(self.K, feature_dim))
        self.u_bank = nn.functional.normalize(self.u_bank)
        self.register_buffer("u_labels", torch.zeros(self.K, num_classes) / num_classes)
        self.register_buffer("ptr", torch.zeros(1, dtype=torch.long))
    
    @torch.no_grad()
    def _update_unlabel_bank(self, feature, prob):
        batch_size = feature.size(0)
        ptr = int(self.ptr[0])
        assert self.K % batch_size == 0
        self.u_bank[ptr:ptr + batch_size] = feature
        self.u_labels[ptr:ptr + batch_size] = prob
        self.ptr[0] = (ptr + batch_size) % self.K

feature_model = MyModel(num_classes=200, feature_dim=256, K=len(train_loader.dataset)//args.batch_size*args.batch_size)

student_proj, student_out = student(images)
teacher_out = student_out.detach()

u_bank   = feature_model.u_bank.clone().detach()
u_labels = feature_model.u_labels.clone().detach()

feature_w, feature_s = student_proj.chunk(2)[0], student_proj.chunk(2)[1]
logits_w, logits_s = student_out.chunk(2)[0], student_out.chunk(2)[1]
prob_w = F.softmax(logits_w, dim=1)

simmatrix_w = feature_w @ u_bank.T
relation_w = F.softmax(simmatrix_w / 0.1, dim=-1)

simmatrix_s = feature_s @ u_bank.T
relation_s = F.softmax(simmatrix_s / 0.1, dim=-1)

kl_div_s_w = F.kl_div(relation_s.log(), relation_w, reduction='batchmean')

nn_qu = relation_s @ u_labels
loss_ne = torch.sum(-nn_qu.log() * prob_w.detach(), dim=1).mean()